<a href="https://colab.research.google.com/github/Mamoro98/Cuda-Programming/blob/main/omer_GPU_Practical4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Omer Kamal Ali Ebead</h1>

# **CUDA Programming on NVIDIA GPUs**

# **Practical 4**

Again make sure the correct Runtime is being used, by clicking on the Runtime option at the top, then "Change runtime type", and selecting an appropriate GPU such as the T4.

Then verify the details of the GPU which is available to you, and upload the usual two header files.

In [1]:
!nvidia-smi

Mon Feb 17 14:53:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   49C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!wget https://people.maths.ox.ac.uk/gilesm/cuda/headers/helper_cuda.h
!wget https://people.maths.ox.ac.uk/gilesm/cuda/headers/helper_string.h


--2025-02-17 14:53:41--  https://people.maths.ox.ac.uk/gilesm/cuda/headers/helper_cuda.h
Resolving people.maths.ox.ac.uk (people.maths.ox.ac.uk)... 129.67.184.129, 2001:630:441:202::8143:b881
Connecting to people.maths.ox.ac.uk (people.maths.ox.ac.uk)|129.67.184.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27832 (27K) [text/x-chdr]
Saving to: ‘helper_cuda.h’

helper_cuda.h       100%[===================>]  27.18K  --.-KB/s    in 0.01s   

2025-02-17 14:53:41 (2.10 MB/s) - ‘helper_cuda.h’ saved [27832/27832]

--2025-02-17 14:53:41--  https://people.maths.ox.ac.uk/gilesm/cuda/headers/helper_string.h
Resolving people.maths.ox.ac.uk (people.maths.ox.ac.uk)... 129.67.184.129, 2001:630:441:202::8143:b881
Connecting to people.maths.ox.ac.uk (people.maths.ox.ac.uk)|129.67.184.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14875 (15K) [text/x-chdr]
Saving to: ‘helper_string.h’

helper_string.h     100%[===================>]  14.53K



---

The next step is to create the file reduction.cu which includes within it a reference C++ routine against which the CUDA results are compared.

# Question 4

In [ ]:
%%writefile reduction.cu

////////////////////////////////////////////////////////////////////////
//
// Practical 4 -- initial code for shared memory reduction for
//                a single block which is a power of two in size
//
////////////////////////////////////////////////////////////////////////

#include <stdlib.h>
#include <stdio.h>
#include <string.h>
#include <math.h>
#include <float.h>

#include <helper_cuda.h>

////////////////////////////////////////////////////////////////////////
// CPU routine
////////////////////////////////////////////////////////////////////////

float reduction_gold(float* idata, int len)
{
  float sum = 0.0f;
  for(int i=0; i<len; i++) sum += idata[i];

  return sum;
}

////////////////////////////////////////////////////////////////////////
// GPU routine
////////////////////////////////////////////////////////////////////////

__global__ void reduction(float *g_odata, float *g_idata)
{
    // dynamically allocated shared memory

    extern  __shared__  float temp[];

    int tid = threadIdx.x;

    // first, each thread loads data into shared memory

    temp[tid] = g_idata[tid];

    // next, we perform binary tree reduction

    for (int d=blockDim.x/2; d>0; d=d/2) {
      __syncthreads();  // ensure previous step completed
      if (tid<d)  temp[tid] += temp[tid+d];
    }

    // finally, first thread puts result into global memory
    if (tid==0) g_odata[0] = temp[0];
}


////////////////////////////////////////////////////////////////////////
// Program main
////////////////////////////////////////////////////////////////////////

int main( int argc, const char** argv)
{
  int num_blocks, num_threads, num_elements, mem_size, shared_mem_size;

  float *h_data, *d_idata, *d_odata;

  // initialise card

  findCudaDevice(argc, argv);

  num_blocks   = 1;  // start with only 1 thread block
  num_threads  = 512;
  num_elements = num_blocks*num_threads;
  mem_size     = sizeof(float) * num_elements;

  // allocate host memory to store the input data
  // and initialize to integer values between 0 and 10

  h_data = (float*) malloc(mem_size);

  for(int i = 0; i < num_elements; i++)
    h_data[i] = floorf(10.0f*(rand()/(float)RAND_MAX));

  // compute reference solution

  float sum = reduction_gold(h_data, num_elements);

  // allocate device memory input and output arrays

  checkCudaErrors( cudaMalloc((void**)&d_idata, mem_size) );
  checkCudaErrors( cudaMalloc((void**)&d_odata, sizeof(float)) );

  // copy host memory to device input array

  checkCudaErrors( cudaMemcpy(d_idata, h_data, mem_size,
                              cudaMemcpyHostToDevice) );

  // execute the kernel

  shared_mem_size = sizeof(float) * num_threads;
  reduction<<<num_blocks,num_threads,shared_mem_size>>>(d_odata,d_idata);
  getLastCudaError("reduction kernel execution failed");

  // copy result from device to host

  checkCudaErrors( cudaMemcpy(h_data, d_odata, sizeof(float),
                              cudaMemcpyDeviceToHost) );

  // check results

  printf("reduction error = %f\n",h_data[0]-sum);
  printf("reduction result = %f\n",h_data[0]);
  printf("reference result = %f\n",sum);

  // cleanup memory

  free(h_data);
  checkCudaErrors( cudaFree(d_idata) );
  checkCudaErrors( cudaFree(d_odata) );

  // CUDA exit -- needed to flush printf write buffer

  cudaDeviceReset();
}


Writing reduction.cu



---

We can now compile and run the executable.  Note that the compilation links in the CUDA random number generation library cuRAND.


In [ ]:
!nvcc reduction.cu -o reduction -I. -lineinfo -arch=sm_70 --ptxas-options=-v --use_fast_math -lcudart

ptxas info    : 0 bytes gmem
ptxas info    : Compiling entry function '_Z9reductionPfS_' for 'sm_70'
ptxas info    : Function properties for _Z9reductionPfS_
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 10 registers, 368 bytes cmem[0]


In [ ]:
!./reduction

GPU Device 0: "Turing" with compute capability 7.5

reduction error = 0.000000
reduction result = 2351.000000
reference result = 2351.000000


**explain why this shows the result is correct, and how the code has performed the required check.**

The program computes the sum in two ways:

1. **Using the CPU (`reduction_gold()`):**  
   - This function computes the sum sequentially using a simple for-loop.

2. **Using the GPU (`reduction()`):**  
   - The CUDA kernel performs **parallel reduction** using **shared memory**.

At the end of execution:
- The **GPU result is copied back to the CPU**.
- The **computed sum is compared** to the **CPU reference sum**.
- If the **difference is zero**, the implementation is correct.


### **1 Compute the CPU Reference Sum**
The CPU computes the **expected correct sum** using:
$$
\text{sum} = \sum_{i=0}^{N-1} \text{idata}[i]
$$

### **2 Copy the GPU Result to the CPU**
$$
\text{cudaMemcpy}(h\_data, d\_odata, \text{sizeof(float)}, \text{cudaMemcpyDeviceToHost})
$$
This retrieves the **GPU result (`d_odata[0]`)** after reduction.

### **3 Compare GPU and CPU Results**
The program prints:
$$
\text{reduction error} = h\_data[0] - \text{sum}
$$
- If the **difference is zero (0.0f)**, the GPU result is correct.


# Question 5

### Round up code

In [ ]:
%%writefile reduction.cu

#include <stdio.h>
#include <stdlib.h>
#include <math.h>

int main()
{
  int m1, m2, m3;

  for (int n=2; n<1029; n++){

    for (m1=1; m1<n; m1=2*m1) {}

    m2 = n-1;
    m2 = m2 | (m2>>1);
    m2 = m2 | (m2>>2);
    // m2 = m2 | (m2>>4);
    // m2 = m2 | (m2>>8);  // this handles up to 16-bit integers
    // m2 = m2 | (m2>>16); // needed to go up to 32-bit integers
    m2 = m2 + 1;

    // in line below need to rename to  __clz() in CUDA; see
    // 1.10 in https://docs.nvidia.com/cuda/cuda-math-api/
    m3 = 1 << (64 - __builtin_clz(n-1));  //  needs n>1

    printf("n, m1, m2, m3 = %2d, %2d, %2d, %2d \n",n,m1,m2,m3);
  }

}


Overwriting reduction.cu


In [ ]:
!nvcc reduction.cu -o reduction -I. -lineinfo -arch=sm_70 --ptxas-options=-v --use_fast_math -lcudart

ptxas info    : 0 bytes gmem


In [ ]:
!./reduction

n, m1, m2, m3 =  2,  2,  2,  2 
n, m1, m2, m3 =  3,  4,  4,  4 
n, m1, m2, m3 =  4,  4,  4,  4 
n, m1, m2, m3 =  5,  8,  8,  8 
n, m1, m2, m3 =  6,  8,  8,  8 
n, m1, m2, m3 =  7,  8,  8,  8 
n, m1, m2, m3 =  8,  8,  8,  8 
n, m1, m2, m3 =  9, 16, 16, 16 
n, m1, m2, m3 = 10, 16, 16, 16 
n, m1, m2, m3 = 11, 16, 16, 16 
n, m1, m2, m3 = 12, 16, 16, 16 
n, m1, m2, m3 = 13, 16, 16, 16 
n, m1, m2, m3 = 14, 16, 16, 16 
n, m1, m2, m3 = 15, 16, 16, 16 
n, m1, m2, m3 = 16, 16, 16, 16 
n, m1, m2, m3 = 17, 32, 31, 32 
n, m1, m2, m3 = 18, 32, 32, 32 
n, m1, m2, m3 = 19, 32, 32, 32 
n, m1, m2, m3 = 20, 32, 32, 32 
n, m1, m2, m3 = 21, 32, 32, 32 
n, m1, m2, m3 = 22, 32, 32, 32 
n, m1, m2, m3 = 23, 32, 32, 32 
n, m1, m2, m3 = 24, 32, 32, 32 
n, m1, m2, m3 = 25, 32, 32, 32 
n, m1, m2, m3 = 26, 32, 32, 32 
n, m1, m2, m3 = 27, 32, 32, 32 
n, m1, m2, m3 = 28, 32, 32, 32 
n, m1, m2, m3 = 29, 32, 32, 32 
n, m1, m2, m3 = 30, 32, 32, 32 
n, m1, m2, m3 = 31, 32, 32, 32 
n, m1, m2, m3 = 32, 32, 32, 32 
n, m1, m

### Extending to handle non-power of 2

In [ ]:
%%writefile reduction.cu

////////////////////////////////////////////////////////////////////////
//
// Practical 4 -- initial code for shared memory reduction for
//                a single block which is a power of two in size
//
////////////////////////////////////////////////////////////////////////

#include <stdlib.h>
#include <stdio.h>
#include <string.h>
#include <math.h>
#include <float.h>

#include <helper_cuda.h>

////////////////////////////////////////////////////////////////////////
// CPU routine
////////////////////////////////////////////////////////////////////////

float reduction_gold(float* idata, int len)
{
  float sum = 0.0f;
  for(int i=0; i<len; i++) sum += idata[i];

  return sum;
}

////////////////////////////////////////////////////////////////////////
// GPU routine
////////////////////////////////////////////////////////////////////////

__global__ void reduction(float *g_odata, float *g_idata)
{
    // dynamically allocated shared memory

    extern  __shared__  float temp[];

    int tid = threadIdx.x;

    // first, each thread loads data into shared memory

    int m3 = 1 << (32 - __builtin_clz(blockDim.x-1));
    m3 = m3/2;
    int diff = blockDim.x - m3;

    if (tid < diff){
      temp[tid] = g_idata[tid];
      temp[tid] += g_idata[tid + m3];
    }
    else {
      temp[tid] = g_idata[tid];
    }



    // next, we perform binary tree reduction

    for (int d=m3/2; d>0; d=d/2) {
      __syncthreads();  // ensure previous step completed
      if (tid<d)  temp[tid] += temp[tid+d];
    }

    // finally, first thread puts result into global memory
    if (tid==0) g_odata[0] = temp[0];
}


////////////////////////////////////////////////////////////////////////
// Program main
////////////////////////////////////////////////////////////////////////

int main( int argc, const char** argv)
{
  int num_blocks, num_threads, num_elements, mem_size, shared_mem_size;

  float *h_data, *d_idata, *d_odata;

  // initialise card

  findCudaDevice(argc, argv);

  num_blocks   = 1;  // start with only 1 thread block
  num_threads  = 192;
  num_elements = num_blocks*num_threads;
  mem_size     = sizeof(float) * num_elements;

  // allocate host memory to store the input data
  // and initialize to integer values between 0 and 10

  h_data = (float*) malloc(mem_size);

  for(int i = 0; i < num_elements; i++)
    h_data[i] = floorf(10.0f*(rand()/(float)RAND_MAX));

  // compute reference solution

  float sum = reduction_gold(h_data, num_elements);

  // allocate device memory input and output arrays

  checkCudaErrors( cudaMalloc((void**)&d_idata, mem_size) );
  checkCudaErrors( cudaMalloc((void**)&d_odata, sizeof(float)) );

  // copy host memory to device input array

  checkCudaErrors( cudaMemcpy(d_idata, h_data, mem_size,
                              cudaMemcpyHostToDevice) );

  // execute the kernel

  shared_mem_size = sizeof(float) * num_threads;
  reduction<<<num_blocks,num_threads,shared_mem_size>>>(d_odata,d_idata);
  getLastCudaError("reduction kernel execution failed");

  // copy result from device to host

  checkCudaErrors( cudaMemcpy(h_data, d_odata, sizeof(float),
                              cudaMemcpyDeviceToHost) );

  // check results

  printf("reduction error = %f\n",h_data[0]-sum);
  printf("reduction result = %f\n",h_data[0]);
  printf("reference result = %f\n",sum);

  // cleanup memory

  free(h_data);
  checkCudaErrors( cudaFree(d_idata) );
  checkCudaErrors( cudaFree(d_odata) );

  // CUDA exit -- needed to flush printf write buffer

  cudaDeviceReset();
}


Overwriting reduction.cu


In [ ]:
!nvcc reduction.cu -o reduction -I. -lineinfo -arch=sm_70 --ptxas-options=-v --use_fast_math -lcudart

ptxas info    : 0 bytes gmem
ptxas info    : Compiling entry function '_Z9reductionPfS_' for 'sm_70'
ptxas info    : Function properties for _Z9reductionPfS_
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 10 registers, 368 bytes cmem[0]


In [ ]:
!./reduction

GPU Device 0: "Turing" with compute capability 7.5

reduction error = 0.000000
reduction result = 896.000000
reference result = 896.000000


# **CUDA Reduction for Non-Power-of-Two Threads**

1. **Find the Largest Power of 2 Less Than `blockDim.x`**
   $$
   m3 = 1 << (32 - \text{__builtin_clz}(\text{blockDim.x} - 1))
   $$
   $$
   m3 = m3 / 2
   $$
   - Computes the largest power of 2 $\leq$ `blockDim.x`.
   - Uses **leading zero count (`__builtin_clz()`)** for fast computation.

2. **Merge Extra Elements Beyond This Power of 2**
   $$
   \text{temp}[tid] = g\_idata[tid]
   $$
   $$
   \text{temp}[tid] += \text{temp}[tid + d]
   $$
   - Threads beyond `m3` are **added** to the corresponding lower-indexed threads.
   - Reduction is performed in a **binary tree fashion**.

4. **Store the Final Result in Global Memory**
   $$
   g\_odata[0] = \text{temp}[0]
   $$
   - **Only thread 0 writes the final sum**.


## **Testing**
- Set `num_threads = 192` (not a power of two).
- Compared GPU result with CPU result (`reduction_gold()`).
- Verified correctness by checking:
  $$
  \text{reduction error} = h\_data[0] - \text{sum}
  $$

Since the **error is zero (~0.0f), the implementation is correct**.


# Question 6

### each block puts its partial sum into a different element of the output array, and then these are transferred to the host and summed there;

In [24]:
%%writefile reduction.cu

////////////////////////////////////////////////////////////////////////
//
// Practical 4 -- initial code for shared memory reduction for
//                a single block which is a power of two in size
//
////////////////////////////////////////////////////////////////////////

#include <stdlib.h>
#include <stdio.h>
#include <string.h>
#include <math.h>
#include <float.h>

#include <helper_cuda.h>

////////////////////////////////////////////////////////////////////////
// CPU routine
////////////////////////////////////////////////////////////////////////

float reduction_gold(float* idata, int len)
{
  float sum = 0.0f;
  for(int i=0; i<len; i++) sum += idata[i];

  return sum;
}

////////////////////////////////////////////////////////////////////////
// GPU routine
////////////////////////////////////////////////////////////////////////

__global__ void reduction(float *g_odata, float *g_idata)
{
    // dynamically allocated shared memory

    extern  __shared__  float temp[];

    int tid = threadIdx.x;
    int global_id = blockIdx.x * blockDim.x + threadIdx.x;

    // first, each thread loads data into shared memory

    temp[tid] = g_idata[global_id];

    // next, we perform binary tree reduction

    for (int d=blockDim.x/2; d>0; d=d/2) {
      __syncthreads();  // ensure previous step completed
      if (tid<d) {
     temp[tid] += temp[tid+d];  // Replace this line with:
    }
    }

    // finally, first thread puts result into global memory
    if (tid==0) g_odata[blockIdx.x] = temp[0];
}


////////////////////////////////////////////////////////////////////////
// Program main
////////////////////////////////////////////////////////////////////////

int main( int argc, const char** argv)
{
  int num_blocks, num_threads, num_elements, mem_size, shared_mem_size;

  float *h_data, *d_idata, *d_odata;

  // initialise card

  findCudaDevice(argc, argv);

  num_blocks   = 1000;  // start with only 1 thread block
  num_threads  = 512;
  num_elements = num_blocks*num_threads;
  mem_size     = sizeof(float) * num_elements;

  // allocate host memory to store the input data
  // and initialize to integer values between 0 and 10

  h_data = (float*) malloc(mem_size);

  for(int i = 0; i < num_elements; i++)
    h_data[i] = floorf(10.0f*(rand()/(float)RAND_MAX));

  // compute reference solution

  float sum = reduction_gold(h_data, num_elements);

  // allocate device memory input and output arrays

  checkCudaErrors( cudaMalloc((void**)&d_idata, mem_size) );
  checkCudaErrors( cudaMalloc((void**)&d_odata, sizeof(float)*num_blocks) );

  // copy host memory to device input array

  checkCudaErrors( cudaMemcpy(d_idata, h_data, mem_size,
                              cudaMemcpyHostToDevice) );

  // execute the kernel

  shared_mem_size = sizeof(float) * num_threads;
  float milli;
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);
  cudaEventRecord(start);
  reduction<<<num_blocks,num_threads,shared_mem_size>>>(d_odata,d_idata);
  cudaEventRecord(stop);
  cudaEventSynchronize(stop);
  cudaEventElapsedTime(&milli, start, stop);
  printf("kernel execution time: %.1f (ms) \n\n", milli);
  getLastCudaError("reduction kernel execution failed");

  // copy result from device to host
  cudaDeviceSynchronize();
  checkCudaErrors( cudaMemcpy(h_data, d_odata, sizeof(float)*num_blocks,
                              cudaMemcpyDeviceToHost) );

  // check results
  float gpu_sum = 0.0f;
  for (int i=0; i<num_blocks; i++){
    gpu_sum += h_data[i];
  }

  printf("reduction error = %f\n",gpu_sum-sum);
  printf("reduction result = %f\n",gpu_sum);
  printf("reference result = %f\n",sum);

  // cleanup memory

  free(h_data);
  checkCudaErrors( cudaFree(d_idata) );
  checkCudaErrors( cudaFree(d_odata) );

  // CUDA exit -- needed to flush printf write buffer

  cudaDeviceReset();
}


Overwriting reduction.cu


In [25]:
!nvcc reduction.cu -o reduction -I. -lineinfo -arch=sm_70 --ptxas-options=-v --use_fast_math -lcudart

ptxas info    : 0 bytes gmem
ptxas info    : Compiling entry function '_Z9reductionPfS_' for 'sm_70'
ptxas info    : Function properties for _Z9reductionPfS_
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 10 registers, 368 bytes cmem[0]


In [26]:
!./reduction

GPU Device 0: "Turing" with compute capability 7.5

kernel execution time: 0.2 (ms) 

reduction error = 0.000000
reduction result = 2305301.000000
reference result = 2305301.000000



**First Solution: Partial Sum per Block**
- Each block **reduces a section of the input** and **writes its sum to `g_odata[blockIdx.x]`**.
- The **host sums all partial sums**.

$$
\text{if} \ (tid == 0), \quad g\_odata[blockIdx.x] = \text{temp}[0]
$$


---


### an atomic addition is used to safely increment a single global sum.

In [27]:
%%writefile reduction.cu

////////////////////////////////////////////////////////////////////////
//
// Practical 4 -- initial code for shared memory reduction for
//                a single block which is a power of two in size
//
////////////////////////////////////////////////////////////////////////

#include <stdlib.h>
#include <stdio.h>
#include <string.h>
#include <math.h>
#include <float.h>

#include <helper_cuda.h>

////////////////////////////////////////////////////////////////////////
// CPU routine
////////////////////////////////////////////////////////////////////////

float reduction_gold(float* idata, int len)
{
  float sum = 0.0f;
  for(int i=0; i<len; i++) sum += idata[i];

  return sum;
}

////////////////////////////////////////////////////////////////////////
// GPU routine
////////////////////////////////////////////////////////////////////////

__global__ void reduction(float *g_odata, float *g_idata)
{
    // dynamically allocated shared memory

    extern  __shared__  float temp[];

    int tid = threadIdx.x;
    int global_id = blockIdx.x * blockDim.x + threadIdx.x;

    // first, each thread loads data into shared memory

    temp[tid] = g_idata[global_id];
    __syncthreads();

    // next, we perform binary tree reduction

    for (int d=blockDim.x/2; d>0; d=d/2) {
      __syncthreads();  // ensure previous step completed
      if (tid<d) {
     temp[tid] += temp[tid+d];
    }
    }

    // finally, first thread puts result into global memory
    if (tid==0) atomicAdd(g_odata,temp[0]) ;
}


////////////////////////////////////////////////////////////////////////
// Program main
////////////////////////////////////////////////////////////////////////

int main( int argc, const char** argv)
{
  int num_blocks, num_threads, num_elements, mem_size, shared_mem_size;

  float *h_data, *d_idata, *d_odata;

  // initialise card

  findCudaDevice(argc, argv);

  num_blocks   = 1000;  // start with only 1 thread block
  num_threads  = 512;
  num_elements = num_blocks*num_threads;
  mem_size     = sizeof(float) * num_elements;

  // allocate host memory to store the input data
  // and initialize to integer values between 0 and 10

  h_data = (float*) malloc(mem_size);

  for(int i = 0; i < num_elements; i++)
    h_data[i] = floorf(10.0f*(rand()/(float)RAND_MAX));

  // compute reference solution

  float sum = reduction_gold(h_data, num_elements);

  // allocate device memory input and output arrays

  checkCudaErrors( cudaMalloc((void**)&d_idata, mem_size) );
  checkCudaErrors( cudaMalloc((void**)&d_odata, sizeof(float)) );

  // copy host memory to device input array

  checkCudaErrors( cudaMemcpy(d_idata, h_data, mem_size,
                              cudaMemcpyHostToDevice) );

  // execute the kernel

  shared_mem_size = sizeof(float) * num_threads;

  float milli;
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);
  cudaEventRecord(start);

  reduction<<<num_blocks,num_threads,shared_mem_size>>>(d_odata,d_idata);
  cudaEventRecord(stop);
  cudaEventSynchronize(stop);
  cudaEventElapsedTime(&milli, start, stop);
  printf("kernel execution time: %.1f (ms) \n\n", milli);

  getLastCudaError("reduction kernel execution failed");

  // copy result from device to host
  cudaDeviceSynchronize();
  float gpu_sum = 0.0f;
  checkCudaErrors(cudaMemcpy(&gpu_sum, d_odata, sizeof(float),
                            cudaMemcpyDeviceToHost));



  printf("reduction error = %f\n",gpu_sum-sum);
  printf("reduction result gpu = %f\n",gpu_sum);
  printf("reference result cpu = %f\n",sum);

  // cleanup memory

  free(h_data);
  checkCudaErrors( cudaFree(d_idata) );
  checkCudaErrors( cudaFree(d_odata) );

  // CUDA exit -- needed to flush printf write buffer

  cudaDeviceReset();
}


Overwriting reduction.cu


In [28]:
!nvcc reduction.cu -o reduction -I. -lineinfo -arch=sm_70 --ptxas-options=-v --use_fast_math -lcudart

ptxas info    : 0 bytes gmem
ptxas info    : Compiling entry function '_Z9reductionPfS_' for 'sm_70'
ptxas info    : Function properties for _Z9reductionPfS_
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 10 registers, 368 bytes cmem[0]


In [29]:
!./reduction

GPU Device 0: "Turing" with compute capability 7.5

kernel execution time: 0.1 (ms) 

reduction error = 0.000000
reduction result gpu = 2305301.000000
reference result cpu = 2305301.000000



**Second Solution: Atomic Addition**
- Each block **computes its partial sum**.
- Instead of storing per-block sums, **an atomic addition updates a global sum**.
$$
\text{if} \ (tid == 0), \quad \text{atomicAdd}(g\_odata, \text{temp}[0])
$$
-no need for host code to do the summation
---



# Question 7

### Modify the block-level reduction to use shuffle instructions as described in Lecture 4. Again your report should include your code, and results to show that the calculation has been carried out successfully

In [9]:
%%writefile reduction.cu

////////////////////////////////////////////////////////////////////////
//
// Practical 4 -- initial code for shared memory reduction for
//                a single block which is a power of two in size
//
////////////////////////////////////////////////////////////////////////

#include <stdlib.h>
#include <stdio.h>
#include <string.h>
#include <math.h>
#include <float.h>

#include <helper_cuda.h>

////////////////////////////////////////////////////////////////////////
// CPU routine
////////////////////////////////////////////////////////////////////////

float reduction_gold(float* idata, int len)
{
  float sum = 0.0f;
  for(int i=0; i<len; i++) sum += idata[i];

  return sum;
}

////////////////////////////////////////////////////////////////////////
// GPU routine
////////////////////////////////////////////////////////////////////////

__global__ void reduction(float *g_odata, float *g_idata)
{
    // dynamically allocated shared memory


    int tid = threadIdx.x;
    int global_id = blockIdx.x * blockDim.x + tid;
    int warpIdx = tid / 32;
    int thread_warp_idx = tid % 32;


    float sum = g_idata[global_id];

    for (int offset = warpSize/2; offset > 0; offset /= 2) {
      sum += __shfl_down_sync(0xFFFFFFFF, sum, offset);
    }

    // Thread 0 writes the final sum
    if (thread_warp_idx == 0) {
        g_odata[warpIdx]= sum;
    }


}


////////////////////////////////////////////////////////////////////////
// Program main
////////////////////////////////////////////////////////////////////////

int main( int argc, const char** argv)
{
  int num_blocks, num_threads, num_elements, mem_size, shared_mem_size;

  float *h_data, *d_idata, *d_odata;

  // initialise card

  findCudaDevice(argc, argv);

  num_blocks   = 1;  // start with only 1 thread block
  num_threads  = 512;
  num_elements = num_blocks*num_threads;
  mem_size     = sizeof(float) * num_elements;

  int num_of_warps = ceil((float) num_threads/32) ;
  printf("number of warps %i \n",num_of_warps);

  // allocate host memory to store the input data
  // and initialize to integer values between 0 and 10

  h_data = (float*) malloc(mem_size);

  for(int i = 0; i < num_elements; i++)
    h_data[i] = floorf(10.0f*(rand()/(float)RAND_MAX));

  // compute reference solution

  float sum = reduction_gold(h_data, num_elements);

  // allocate device memory input and output arrays

  checkCudaErrors( cudaMalloc((void**)&d_idata, mem_size) );
  checkCudaErrors( cudaMalloc((void**)&d_odata, sizeof(float)*num_of_warps) );

  // copy host memory to device input array

  checkCudaErrors( cudaMemcpy(d_idata, h_data, mem_size,
                              cudaMemcpyHostToDevice) );

  // execute the kernel

  shared_mem_size = sizeof(float) * num_threads;
  reduction<<<num_blocks,num_threads,shared_mem_size>>>(d_odata,d_idata);
  getLastCudaError("reduction kernel execution failed");

  // copy result from device to host
  cudaDeviceSynchronize();
  checkCudaErrors( cudaMemcpy(h_data, d_odata, sizeof(float)*num_of_warps,
                              cudaMemcpyDeviceToHost) );

  // check results
  float gpu_sum = 0.0f;
  for (int i=0; i<num_of_warps; i++){
    gpu_sum += h_data[i];
  }

  printf("reduction error = %f\n",gpu_sum-sum);
  printf("reduction result = %f\n",gpu_sum);
  printf("reference result = %f\n",sum);

  // cleanup memory

  free(h_data);
  checkCudaErrors( cudaFree(d_idata) );
  checkCudaErrors( cudaFree(d_odata) );

  // CUDA exit -- needed to flush printf write buffer

  cudaDeviceReset();
}


Overwriting reduction.cu


In [10]:
!nvcc reduction.cu -o reduction -I. -lineinfo -arch=sm_70 --ptxas-options=-v --use_fast_math -lcudart

ptxas info    : 0 bytes gmem
ptxas info    : Compiling entry function '_Z9reductionPfS_' for 'sm_70'
ptxas info    : Function properties for _Z9reductionPfS_
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 10 registers, 368 bytes cmem[0]


In [11]:
!./reduction

GPU Device 0: "Turing" with compute capability 7.5

number of warps 16 
reduction error = 0.000000
reduction result = 2351.000000
reference result = 2351.000000


**Replacing Shared Memory with Warp Shuffle**
- Each thread **loads its value** into a register:
  $$
  \text{float sum} = g\_idata[\text{global_id}]
  $$
- Instead of using **shared memory and `__syncthreads()`**, warp shuffle **propagates values downwards**:
  $$
  \text{sum} += \_\_shfl\_down\_sync(0xFFFFFFFF, \text{sum}, \text{offset})
  $$
  - This efficiently reduces values within a **warp (32 threads)**.

---

**Storing the Final Sum per Warp**
- Only **thread 0 in each warp** stores the **warp sum**:
  $$
  \text{if} (\text{thread_warp_idx} == 0) \quad g\_odata[\text{warpIdx}] = \text{sum}
  $$
  - **`thread_warp_idx == 0` ensures that only one thread per warp writes.**
  - This stores **one sum per warp** in `g_odata`.

---

**Why This Approach is More Efficient**
- **Faster than shared memory** because data stays in registers.
- **Avoids `__syncthreads()` overhead**, making execution more efficient.
- **Uses warp-wide reduction**, minimizing memory transactions.

**Limitations**
- Works **only within a warp (32 threads)**. If we need block-wide reduction, an extra step is required.
- Final summation across warps must be done separately (either in the host or with another kernel).

---

**Testing**
- **Set `num_threads = 512`** → Each block has **512 threads = 16 warps**.
- **Computed number of warps:**
  $$
  \text{num_of_warps} = \lceil \frac{512}{32} \rceil = 16
  $$
- **Final sum computed on the host**:
  $$
  \text{gpu_sum} = \sum_{i=0}^{\text{num_of_warps}-1} h\_data[i]
  $$
- **Compared with CPU reference (`reduction_gold()`):**
  $$
  \text{reduction error} = h\_data[0] - \text{sum}
  $$
  - If the error is **zero (0.0f)**, the calculation is **correct**.

---


# Extending to multiple blocks

In [49]:
%%writefile reduction.cu

////////////////////////////////////////////////////////////////////////
//
// Practical 4 -- initial code for shared memory reduction for
//                a single block which is a power of two in size
//
////////////////////////////////////////////////////////////////////////

#include <stdlib.h>
#include <stdio.h>
#include <string.h>
#include <math.h>
#include <float.h>

#include <helper_cuda.h>

////////////////////////////////////////////////////////////////////////
// CPU routine
////////////////////////////////////////////////////////////////////////

float reduction_gold(float* idata, int len)
{
  float sum = 0.0f;
  for(int i=0; i<len; i++) sum += idata[i];

  return sum;
}

////////////////////////////////////////////////////////////////////////
// GPU routine
////////////////////////////////////////////////////////////////////////

__global__ void reduction(float *g_odata, float *g_idata)
{
    extern __shared__ float shared_mem[];

    int tid = threadIdx.x;
    int global_id = blockIdx.x * blockDim.x + tid;
    int warpIdx = tid / 32;
    int thread_warp_idx = tid % 32;

    float sum = g_idata[global_id];

    for (int offset = warpSize / 2; offset > 0; offset /= 2) {
        sum += __shfl_down_sync(0xFFFFFFFF, sum, offset);
    }

    if (thread_warp_idx == 0) {
        shared_mem[warpIdx] = sum;
    }
    __syncthreads();

    if (warpIdx == 0) {
        sum = (tid < blockDim.x / warpSize) ? shared_mem[tid] : 0.0f;
        for (int offset = warpSize / 2; offset > 0; offset /= 2) {
            sum += __shfl_down_sync(0xFFFFFFFF, sum, offset);
        }

        if (tid == 0) {
            g_odata[blockIdx.x] = sum;
        }
    }
}



////////////////////////////////////////////////////////////////////////
// Program main
////////////////////////////////////////////////////////////////////////

int main( int argc, const char** argv)
{
  int num_blocks, num_threads, num_elements, mem_size, shared_mem_size;

  float *h_data, *d_idata, *d_odata;

  // initialise card

  findCudaDevice(argc, argv);

  num_blocks   = 128;  // start with only 1 thread block
  num_threads  = 512;
  num_elements = num_blocks*num_threads;
  mem_size     = sizeof(float) * num_elements;

  int num_of_warps = num_blocks * (num_threads/32) ;
  printf("number of warps %i \n",num_of_warps);

  // allocate host memory to store the input data
  // and initialize to integer values between 0 and 10

  h_data = (float*) malloc(mem_size);

  for(int i = 0; i < num_elements; i++)
    h_data[i] = floorf(10.0f*(rand()/(float)RAND_MAX));

  // compute reference solution

  float sum = reduction_gold(h_data, num_elements);

  // allocate device memory input and output arrays

  checkCudaErrors( cudaMalloc((void**)&d_idata, mem_size) );
  checkCudaErrors( cudaMalloc((void**)&d_odata, sizeof(float)*num_blocks) );

  // copy host memory to device input array

  checkCudaErrors( cudaMemcpy(d_idata, h_data, mem_size,
                              cudaMemcpyHostToDevice) );

  // execute the kernel

  shared_mem_size = sizeof(float) * num_threads;

  reduction<<<num_blocks,num_threads,shared_mem_size>>>(d_odata,d_idata);
  getLastCudaError("reduction kernel execution failed");

  // copy result from device to host
  cudaDeviceSynchronize();
  checkCudaErrors( cudaMemcpy(h_data, d_odata, sizeof(float)*num_blocks,
                              cudaMemcpyDeviceToHost) );

  // check results
  float gpu_sum = 0.0f;
  for (int i=0; i<num_blocks; i++){
    gpu_sum += h_data[i];
  }

  printf("reduction error = %f\n",gpu_sum-sum);
  printf("reduction result = %f\n",gpu_sum);
  printf("reference result = %f\n",sum);

  // cleanup memory

  free(h_data);
  checkCudaErrors( cudaFree(d_idata) );
  checkCudaErrors( cudaFree(d_odata) );

  // CUDA exit -- needed to flush printf write buffer

  cudaDeviceReset();
}


Overwriting reduction.cu


In [50]:
!nvcc reduction.cu -o reduction -I. -lineinfo -arch=sm_70 --ptxas-options=-v --use_fast_math -lcudart

ptxas info    : 0 bytes gmem
ptxas info    : Compiling entry function '_Z9reductionPfS_' for 'sm_70'
ptxas info    : Function properties for _Z9reductionPfS_
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 12 registers, 368 bytes cmem[0]


In [51]:
!./reduction

GPU Device 0: "Turing" with compute capability 7.5

number of warps 2048 
reduction error = 0.000000
reduction result = 294165.000000
reference result = 294165.000000



- Each **warp sum is stored in shared memory** before final block-wide reduction.

- Instead of **storing per warp**, only **one final sum per block** is written:
  $$
  g\_odata[\text{blockIdx.x}] = \text{final block sum}
  $$
  
- The final summation step **properly accumulates all warp sums**.

---

## **Algorithm**
1. **Each thread loads data from global memory:**
   $$
   \text{sum} = g\_idata[\text{global_id}]
   $$

2. **First reduction within each warp using `__shfl_down_sync()`:**
   $$
   \text{sum} += \_\_shfl\_down\_sync(0xFFFFFFFF, \text{sum}, \text{offset})
   $$

3. **Thread 0 of each warp writes to shared memory:**
   $$
   \text{if} \ (\text{thread\_warp\_idx} == 0) \quad \text{shared\_mem}[\text{warpIdx}] = \text{sum}
   $$

4. **Final reduction across warps using warp shuffle:**
   $$
   \text{sum} = \text{shared\_mem}[\text{tid}]
   $$
   $$
   \text{sum} += \_\_shfl\_down\_sync(0xFFFFFFFF, \text{sum}, \text{offset})
   $$

5. **Only thread 0 writes final block sum to global memory:**
   $$
   \text{if} \ (\text{tid} == 0) \quad g\_odata[\text{blockIdx.x}] = \text{sum}
   $$

---



# Un-Assign



---
By going back to the previous code block you can modify the code to complete the initial Practical 4 exercises. Remember to first make your own copy of the notebook so that you are able to edit it.

For the first exercise, it may be useful to know that the following line of code will round up the input n to the nearest power of 2, so then dividing it by 2 gives the largest power of 2 less than n.

`for (m=1; m<n; m=2*m) {} `

For students doing this as an assignment to be assessed, you should again add your name to the title of the notebook (as in "Practical 4 -- Mike Giles.ipynb"), make it shared (see the Share option in the top-right corner) and provide the shared link as the submission mechanism.



In [ ]:
from google.colab import runtime
runtime.unassign()